# A Hybrid Learning Approach to Generating Synthetic Positions for Tax Loss Harvesting

# Stock Market Backtester

In [24]:
import pandas as pd
import yfinance as yf
import pyfolio as pf #install using  pip3 install git+https://github.com/quantopian/pyfolio
from enum import Enum
%matplotlib inline

In [25]:
start_date = "2012-01-09" #TODO choose real start date
end_date = "2013-01-10" #TODO choose real end date
all_tickers = ['SPY', 'AAPL'] #TODO make list of real tickers

# Price history downloaded from Yahoo Finance
price_history = {}
for symbol in all_tickers:
    price_history[symbol] = yf.Ticker(symbol).history(start = start_date, end = end_date)

# Gets a list of all valid trading dates
all_dates = price_history[all_tickers[0]].axes[0].values
dates = pd.DatetimeIndex(data = all_dates)
current_date = all_dates[0]

In [26]:
def get_stock_value(ticker, day, time) -> float:
    """Retrieves the closing price of a stock on a given date
    :param ticker: stock for which to retrieve the value
    :param day: date on which to retrieve the value
    :param time of day to calculate the value, either Time.OPEN or Time.CLOSE
    """
    if time==Time.OPEN:
        return price_history[ticker].Open[day]
    elif time==Time.CLOSE:
        return price_history[ticker].Close[day]

class Time(Enum):
    OPEN = 0
    CLOSE = 1

class Portfolio:
    def __init__(self, starting_cash_balance: float = 1000000, starting_stocks=None):
        """Creates a new portfolio
        :param starting_cash_balance: The amount of cash the portfolio should start with (default 100,000.0)
        :param starting_stocks: A dictionary containing the stocks the portfolio should begin with {'Ticker': Float Quantity} (default no starting stocks)"""
        if starting_stocks is None:
            starting_stocks = {}
        self.stocks = starting_stocks
        self.cash_balance = starting_cash_balance
        self.value = 0
        self.calculate_value(Time.OPEN)
        self.closing_prices = []
        self.returns = None

    def calculate_value(self, time):
        """Calculates the value of the portfolio based on the close of the global current date
        :param time of day to calculate the value, either Time.OPEN or Time.CLOSE
        """
        self.calculate_value_date(current_date, time)

    def calculate_value_date(self, day, time):
        """Calculates the value of the portfolio based on a specified date
        :param day: the date on which the portfolio's value should be calculated
        :param time of day to calculate the value, either Time.OPEN or Time.CLOSE
        """
        self.value = self.cash_balance
        for stock in self.stocks.keys():
            self.value += get_stock_value(stock, day, time) * self.stocks[stock]

    def end_day(self):
        """"Calculates the day's closing value and adds it to a list
        """
        self.calculate_value(Time.CLOSE)
        self.closing_prices.append(self.value)

    def end_simulation(self):
        """Creates a Pandas Series of percent change of closing values and returns it"""
        self.returns = pd.Series(data = self.closing_prices, index = dates).pct_change()
        return self.returns




In [27]:
portfolio = Portfolio(0, {'SPY': 1})
for date in all_dates:
    # update the current day
    current_date = date

    # calculate portfolio value on the opening of this day
    portfolio.calculate_value(Time.OPEN)



    # ALlow the portfolio to perform end-of-day updates
    portfolio.end_day()

# Inform the portfolio that the simulation has ended
returns = portfolio.end_simulation()
print(returns)

2012-01-09         NaN
2012-01-10    0.008670
2012-01-11    0.000542
2012-01-12    0.002399
2012-01-13   -0.005174
                ...   
2013-01-03   -0.002259
2013-01-04    0.004392
2013-01-07   -0.002733
2013-01-08   -0.002877
2013-01-09    0.002542
Length: 252, dtype: float64


In [28]:
#pf.create_returns_tear_sheet(returns)